In [1]:
import torch
from schnetpack.datasets import QM9
import schnetpack as spk
import os
from my_config import config_args
from Model.HGDM import HyperbolicAE,HyperbolicDiffusion
import optimizers
import numpy as np
import logging
import time
from tqdm import tqdm
from Model import Encoders, Decoders

In [2]:
qm9data = QM9('./data/qm9.db', download=True,load_only=[QM9.U0])
qm9split = './data/qm9split'
print(len(qm9data))

train, val, test = spk.train_test_split(
        data=qm9data,
        num_train=20000,
        num_val=10000,
        split_file=os.path.join(qm9split, "split20000-10000.npz"),
    )
print(len(train),len(val),len(test))

train_loader = spk.AtomsLoader(train, batch_size=256, shuffle=False)
val_loader = spk.AtomsLoader(val, batch_size=256)

133885
20000 10000 103885


In [3]:
import json
class obj(object):
    def __init__(self, dict_):
        self.__dict__.update(dict_)
args = json.loads(json.dumps(config_args), object_hook=obj)

In [4]:
encoder_path = './saved_model/'+args.model+'-encoder_KL.pt'
decoder_path = './saved_model/'+args.model+'-decoder_KL.pt'

encoder = getattr(Encoders, args.model)(args)
encoder.load_state_dict(torch.load(encoder_path))
if args.model == 'MLP':
    decoder = Decoders.model2decoder[args.model](None, args)
else:
    decoder = Decoders.model2decoder[args.model](encoder.curvatures, args)
decoder.load_state_dict(torch.load(decoder_path))

device = torch.device('cuda')
model = HyperbolicDiffusion(args,encoder,decoder)
model = model.to(device)

In [5]:
optimizer = getattr(optimizers, args.optimizer)(params=model.parameters(), lr=args.lr,
                                                    weight_decay=args.weight_decay)
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=args.lr_reduce_freq,
    gamma=float(args.gamma)
)
tot_params = sum([np.prod(p.size()) for p in model.parameters()])
print(f"Total number of parameters: {tot_params}")

# Train model
t_total = time.time()

Total number of parameters: 1625833


In [6]:

for epoch in range(args.epochs):
    model.train()
    counter = 0
    loss_sum,n = 0.0,0
    for input in tqdm(train_loader):
        for key in input:
            input[key] = input[key].to(torch.device('cuda'))
        t = time.time()
        model.train()
        optimizer.zero_grad()
        loss = model(input).sum()
        print(loss,lr_scheduler.get_last_lr())
        loss.backward()
        loss_sum += loss
        n += 1
        # if args.grad_clip is not None:
        #     grad_clip = float(args.grad_clip)
        #     all_params = list(model.parameters())
        #     for param in all_params:
        #         torch.nn.utils.clip_grad_value_(param, grad_clip)
        optimizer.step()
        lr_scheduler.step()
    if (epoch + 1) % args.log_freq == 0:
        str = " ".join(['Epoch: {:04d}'.format(epoch + 1),
                               'lr: {}'.format(lr_scheduler.get_last_lr()[0]),
                               'loss: {:.4f}'.format(loss_sum/n),
                               'time: {:.4f}s'.format(time.time() - t)
                               ])
        print(str)

    model.eval()
    with torch.no_grad():
        loss_sum,n = 0.0,0
        for input in tqdm(val_loader):
            for key in input:
                input[key] = input[key].to(torch.device('cuda'))
            t = time.time()
            model.train()
            optimizer.zero_grad()
            loss = model(input).sum()
            n += 1
            loss_sum += loss

        print('val_loss:',loss_sum.item()/n)


  1%|▏         | 1/79 [00:02<02:38,  2.03s/it]

tensor([0.0000, 0.0000, 3.3260, 0.5211, 1.7913, 0.0000, 2.1762, 3.1931, 3.4874,
        1.9030, 1.9651, 0.0000, 0.0000, 0.2339, 2.6870, 1.8458, 0.0000, 1.6993,
        4.9039, 0.0000], device='cuda:0', grad_fn=<SelectBackward0>)
tensor(40.4763, device='cuda:0', grad_fn=<SumBackward0>) [0.0001]


  3%|▎         | 2/79 [00:02<01:19,  1.03s/it]

tensor([0.0000, 0.0000, 1.0696, 0.0000, 1.7869, 0.0000, 2.7751, 0.8385, 4.9359,
        0.4971, 2.7933, 0.0000, 0.0000, 1.1113, 2.8633, 1.9674, 0.0000, 0.6262,
        2.1722, 0.0000], device='cuda:0', grad_fn=<SelectBackward0>)
tensor(13.8018, device='cuda:0', grad_fn=<SumBackward0>) [0.0001]


  4%|▍         | 3/79 [00:02<00:54,  1.39it/s]

...

 30%|███       | 24/79 [00:11<00:24,  2.26it/s]

tensor([0.0000, 0.0000, 0.5302, 0.9593, 0.6913, 1.0002, 0.5685, 4.5233, 1.8843,
        0.2840, 0.2117, 0.0000, 0.0000, 0.0000, 3.1614, 1.1514, 0.0000, 0.0000,
        4.1048, 0.0000], device='cuda:0', grad_fn=<SelectBackward0>)
tensor(1.1485, device='cuda:0', grad_fn=<SumBackward0>) [6.400000000000001e-05]


 30%|███       | 24/79 [00:11<00:26,  2.07it/s]

tensor([0.0000, 0.0000, 0.1373, 0.7449, 0.6431, 1.3953, 0.2541, 4.2558, 1.6054,
        0.3159, 0.0000, 0.0000, 0.0000, 0.0000, 3.1002, 1.3779, 0.0000, 0.0000,
        3.8338, 0.0000], device='cuda:0', grad_fn=<SelectBackward0>)
tensor(1.0969, device='cuda:0', grad_fn=<SumBackward0>) [6.400000000000001e-05]


KeyboardInterrupt: 